# Algebras and Recursion

First, let's present an example of how to codify algebraic expressions (+, -, *)
for our own datatypes.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAP/PhDThesis/notes/FunctionalProgrammingCategoryTheory/julia`


## 1. F-Algebras

Remember our definition of a monoid, which was $(M, \eta, \mu)$, where:

$$
\eta: 1 \to M
$$

$$
\mu: M \times M \to M
$$

We can actually turn this definition into something more abstract.
Note, a monoid consists of picking an object $M$ and two
function $\eta \in M^{1}$ and $\mu \in M^{M \times M}$. To select
two functions one in $M^1$ and another in $M^{M \times M}$ is the same
as picking one function in $M^1 \times M^{M \times M}$, which is equal
to picking one function in $M^{1 + M \times M}$.

Hence, a monoid is a function of $M^{1 + M \times M}$ where the function
satisfies the associativity and neutrality conditions. In other words,
every function of $M^{1 + M \times M}$ is a *potential* monoid.

The nice thing about this abstraction is that we can now conceptualize other
structures. Consider for example a group in abstract algebra. A group is a monoid
plus an inverse to every element. This inverse is then a function that for every
element $M$ returns another element $M$. Therefore, if a potential monoid
is a function of $M^{1 + M \times M}$, a potential group is a function
of $M^{1 + M\times M + M}$... Where did this comes from? Just follow the same
logic as in the case of the monoid. We have a set $M$ together with three
functions, one in $M^1$, another in $M^{M \times M}$ and a new one in
$M^M$. Taking the cartesian product we have:

$$
M^1 \times M^{M \times M} \times M^M = M^{1 + M\times M + M}.
$$


We can take this idea even further. Remeber,
the sum and product of objects are actually functors. Hence,
the algebraic expression $1 + M\times M + M$ is also a functor.
Thus, every algebraic expression can be generalized as a functor

We can now say, for example, that a monoid over set (or type) $T$
is a function from functor $((\cdot \times \cdot) + 1)(T)$ to $T$.

All these things in abstract algebra (e.g. monoids, groups, rings)
can now be seeing as functions from $Fa$ to $a$, where $a$ is a set,
and $F$ is a functor.

This gives rise to our definition of an F-algebra.

**Definition (F-Algebra)**: Let $F:\mathcal C \to \mathcal C$ be an endofunctor. An $F$-algebra $(\phi,a)$ is

1) An object $a \in \mathcal C$, called *carrier*;
2) A morphism $\phi : Fa \to a$, called *structure map*.





## 2. Example 1 - Ring

Let's implement an example.

In [2]:
abstract type RingF{T} end

RingF(::Type{T}) where T = RingF{T}

struct ZeroF{T} <: RingF{T} end
struct OneF{T} <: RingF{T} end
struct AddF{T} <: RingF{T}
    _1::T
    _2::T
end
struct MulF{T} <: RingF{T}
    _1::T
    _2::T
end

Note that `RingF{T}` is a parametric type in julia, and we define a function `RingF(::Type{T})`, where this function
takes a type `T` and returns a `RingF{T}` type, i.e. this is similar to a functor, where it takes a type (object in our
category) and returns another type.

For `RingF` to be indeed a functor, we need to define `RingF(f::Function)`, i.e. a way where a function
`f(x::T)` becomes `RingF(f)(RingF(x))`.

Remember the previous section. This implementation is done via a `fmap` function.

First, let's try using the `Functors` package.

In [3]:
RingF(f::Function, x::ZeroF{T}) where T =  ZeroF
using Functors

@functor ZeroF{T} where T
@functor OneF{T} where T
@functor AddF{T} where T

In [4]:
x = AddF{Int}(1,2)
fmap(string, x)

AddF{String}("1", "2")

In [5]:
fmap(string, ZeroF{Int}())

"ZeroF{Int64}()"

Note that the first case with `Add{Int}` the `fmap` worked as expected. But not in the `ZeroF{Int}`.
For the second case, we expected `ZeroF{String}`. But why didn't it work?

It didn't work because Julia does not consider the domain and codmain of a function as type parameters. This means
that we don't know exaclty what type a function `f` will return. And since `ZeroF` has not values inside,
we cannot apply our function to it and get a type back. So how do we solve this?

We actually already did this when we talked about morphisms. Hence, let's use the `FunctionWrappers.jl`
package that behaves like our Morphism construction.

In [6]:
import FunctionWrappers: FunctionWrapper

In [7]:
wstring = FunctionWrapper{Real,Tuple{String}}(x->string(x))

f_map(f::FunctionWrapper, x::ZeroF{Int}) = ZeroF{typeof(f).parameters[2].parameters[1]}()

f_map (generic function with 1 method)

In [8]:
f_map(wstring, ZeroF{Int}())

ZeroF{String}()

Hence we are able to construct a functor `RingF`.
**Note that we are assuming that** `ZeroF`, `OneF` and `AddF` are all the possible subtypes of `RingF`. This is actually
not the case in Julia. Since our `RingF` is an abstract type, it can have more subtypes under it. Hence, our functor is
properly defined only as long as we are tackling all existing subtypes of our `RingF`.

But what about the F-Algebra? Consider the code below:

In [9]:
evalZ(e::ZeroF)::Int = 0
evalZ(e::OneF)::Int  = 1
evalZ(e::AddF)  = evalZ(e._1) + evalZ(e._2)
evalZ(e::MulF) = evalZ(e._1) * evalZ(e._2)

evalZ (generic function with 4 methods)

So, we've defined a type called `RingF` which is a functor, and we've defined
some operations on it. These operations were instantiated via a function called `evalZ`.

In our example, `Int` is the carrier, `evalZ` is the structure map, and `RingF` is the functor.
In other words, (`evalZ`, `Int`) is a `RingF`-algebra.

## 3. Recursive Data Type


The example above was limited. The `RingF` had only one level of depth. Suppose now we wanted to create a recursive
structure, where we could operate on our subtypes... Check the example below.

Let's define a type "Expression" that has as subtypes the possible
algebraic operations, and the neutral element for both the sum and the multiplication
operations.

In [10]:
abstract type Expression end

struct RZero <: Expression end
struct ROne <: Expression end
struct RAdd <: Expression
    _1::Expression
    _2::Expression
end
struct RMult <: Expression
    _1::Expression
    _2::Expression
end
struct RNeg <: Expression
    _1::Expression
end



Note that we've used structs to encode sums, and multiplications, instead of defining
a functions.

Yet, our struct does not receive the actual values, but other subtypes.
Hence, we have redefined the whole algebra of summation and multiplication
in an abstract manner, i.e. without passing the actual numbers, but just
structs.

In [11]:
RAdd(ROne(),RZero())

RAdd(ROne(), RZero())

The expression above is purely abstract. 

Let's now do an actual example, by instantiating our types. This means that
we are going to specify how our sum and multiplication work.

We'll do the most basic example first, by defining RZero to be 0, and ROne to be 1.

In [12]:
evalZ(e::RZero) = 0
evalZ(e::ROne)  = 1
evalZ(e::RAdd)  = evalZ(e._1) + evalZ(e._2)
evalZ(e::RMult) = evalZ(e._1) * evalZ(e._2)
evalZ(e::RNeg)  = -evalZ(e._1)

evalZ (generic function with 9 methods)

In [13]:
evalZ(RAdd(RZero(),
        RMult(ROne(),
            RAdd(ROne(),
                RAdd(RZero(),
                    ROne())))))

2

Let's create two more possibilities. A constant and a variable:

In [14]:
struct RVar <: Expression
    _1::String
end

struct RConst <: Expression
    _1::Real
end

In [15]:
# 2 x^2 + 3 x + 4
ex = RAdd(  RMult(
                RConst(2),
                (RMult( RVar("x"),
                        RVar("x")))),
            RAdd(RMult(RConst(3),RVar("x")),RConst(4)))

# Defina que a variavel "x" é 2, e 0 caso RVar seja outra variável.
evalZ(e::RVar) = e._1 == "x" ? 2 : 0
evalZ(e::RConst) = e._1
evalZ(ex)

18

In [16]:
2 * 2^2 + 3*2 + 4

18

### Some technicallities...

So we've shown how a infinite recursive type can be created and worked on. Yet, what kind of mathematical structure is this?
It can actually be shown that such recursive structure is nothing more than an F-Algebra.

The infinite recursion can be shown to give origin to a fixed point, which is equivalent
to applying the functor an infinite amount of times. This fixed point represents the initial case.

Lastly, we show another example of such infinite recursion... The natural numbers.

The algebra for natural numbers is given by (`Int`, `f`), where `f` is a function
from $N + 1 \to N$. Why? Because we have to define a pair of functions,

$$
zero: 1 \to \mathbb N
$$

$$
succ: \mathbb N \to \mathbb N.
$$

In [17]:
abstract type ℕ end

struct Zero <: ℕ end
struct Succ <: ℕ
    _1::ℕ
end

In [18]:
evalZ(x::Zero) = 0
evalZ(x::Succ) = evalZ(x._1) + 1

evalZ (generic function with 13 methods)

In [44]:
evalZ((Succ ∘ Succ ∘ Zero)())

2

## 4. Category of F-Algebras

We have defined an F-algebra as $(a,f)$ where $a$ is the carrier object (e.g. a type  `Int` in the case of programming)
and a strucutral map $f:Fa\to a$ which would follow a set of conditions.

It can be shown that the F-Algebras form a category, where an algebra $(a,f)$ is an object, and homomorphisms between the algebras are morphisms.
Hence, a morphism would be a map from $(a,f )$ to $(b, g)$.

A result known as Lambek's Theorem states that if an F-algebra has an initial object $(i,\phi)$,
then $\phi: F i \to i$ is an isomorphism, i.e. $F i \cong i$.
This means that $i$ is a fixed point of $F$, since $F Fi \cong F i \cong i$.


## 5 Catamorphism

## 6 Folds

Another important recursive abstraction is what's called a fold.

Note that some recursive functions are similar:

In [45]:
sum_recursive(x::Vector{<:Real}) = length(x) == 0 ? 0 : +(x[begin], sum_recursive(x[begin+1:end]))
sum_recursive([1,3,4])

8

In [46]:
concat_recursive(x::Vector{<:String}) = length(x) == 0 ? "" : *(x[begin], concat_recursive(x[begin+1:end]))
concat_recursive(["t","e","s","t"])

"test"

In [47]:
map_recursive(f::Function, x::Vector) = length(x) == 0 ? [] : vcat(f(x[begin]), map_recursive(f, x[begin+1:end]))
map_recursive(x->x^2, [1,2,3,4,5])

5-element Vector{Any}:
  1
  4
  9
 16
 25

All of them have a base case (e.g. length(x) == 0), and all of them have some operation that "merges" the results. 

This is begging us to define an abstraction, i.e. how can we define all these functions by providing only
the part of them is is changing.
This abstraction is a **Foldr**. The letter "r" in the end means that we are "folding" from the right.
We can also fold from the left, which will give us the foldl. Yet, we'll start with foldr.


Note that, in each case, we have a monoids. Remember, a moinoid is a triple $(M,e,*)$, where $M$ is a set, $e \in M$ is the
unit element and $*:M\times M \to M$. For $m,n,p \in M$,
* $e * m = m$,
* $m * e = m$,
* $(m * n) * p = m * (n * p)$

In our examples, we have (`Vector{<:Real}`, `0`, `+`), (`Vector{<:String}`, `""`, `*`).
The `map_recursive` is a bit more complicated. Our function is not `vcat(x,y)`, but `vcat(f(x),y)`.
We can circumvent this by defining a function `vcat_map(f::Function) = (x,y) -> vcat(f(x),y)`. Thus,
we get the monoid (`Vector`, `[]`, `vcat_map(f)`).

Below we implement our fold.

In [48]:
foldr(x::M, e, f) where M = length(x) == 0 ? e : f(x[begin], foldr(x[begin+1:end], e, f))

foldr (generic function with 1 method)

Finally, let's show how this indeed generalizes our implementations. 

In [49]:
foldr([1,2,3], 0, +)

6

In [50]:
foldr(["a","b", "c"], "", *)

"abc"

In [51]:
vcat_map(f::Function) = (x,y) -> vcat(f(x),y)
foldr([1,2,3], [], vcat_map(x->x^2))

3-element Vector{Any}:
 1
 4
 9

**Beautiful!**

Now, what exactly is foldr doing? It's simple, consider the following pattern:

foldr (@) e [w,x,y,z] = w @ (x @ (y @ (z @ e)))


Hence, the foldr is applying the `@` operation from right to left. We can now understand why
a foldl is possible:

foldl (@) e [w,x,y,z] = (((e @ w) @ x) @ y) @ z

In [52]:
foldl(x::M, e, f) where M = length(x) == 0 ? e : f(x[end], foldr(x[begin:end-1], e, f))
foldl(["a","b", "c"], "", *)

"cab"

Hence, our left fold can easily be used to define things like reversing concatenation. 